# CloudWatch

In [26]:
%%time
import boto3
import pandas as pd
from datetime import datetime, timedelta, timezone
import io
import json
from time import sleep
from threading import Thread

import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.huggingface.model import HuggingFaceModel

CPU times: user 43 µs, sys: 0 ns, total: 43 µs
Wall time: 45.5 µs


In [19]:
##S3 prefixes
bucket_name = 'aai-540-final-data'
region_name = 'us-west-2'
session = sagemaker.Session(boto3.Session(region_name=region_name))
s3 = session.boto_session.client('s3')
s3_path = 'athena/results/'

prefix = "sagemaker/AIEmotions-ModelQualityMonitor"
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket_name}/{data_capture_prefix}"


ground_truth_upload_path = (
    f"s3://{bucket_name}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket_name}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region_name)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Image URI: 159807026194.dkr.ecr.us-west-2.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/datacapture
Ground truth path: s3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-16-21-56-34
Report path: s3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/reports


In [4]:
#Retrieve Testing Data 
featurestore_runtime = session.boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region_name)
feature_group_name = 'emotion_feature_group_13_03_13_24_1707794028'

athena_client = boto3.client('athena', region_name=region_name)
query_string = f"""
SELECT * FROM "{feature_group_name}"
WHERE data_type = 'test'
"""
output_location = f's3://{bucket_name}/athena/results/'

response_test = athena_client.start_query_execution(
    QueryString=query_string,
    QueryExecutionContext={
        'Database': 'sagemaker_featurestore'
    },
    ResultConfiguration={
        'OutputLocation': output_location,
    }
)
test_location_id = response_test['QueryExecutionId']

In [5]:
def wait_for_query_completion(client, query_execution_id):
    while True:
        # Get the query execution status
        response = client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        
        # If the query is finished, break from the loop
        if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            return status
        
        # Otherwise, wait a bit before checking again
        time.sleep(5)

In [14]:
query_status_test = wait_for_query_completion(athena_client, test_location_id)

if query_status_test == 'SUCCEEDED':
    # Construct the S3 key for the query results
    s3_test_location = s3_path + test_location_id+ ".csv"
    
    # Now that the query has succeeded, we can safely access the results
    test_data_obj = s3.get_object(Bucket=bucket_name, Key=s3_test_location)

    df_test = pd.read_csv(io.BytesIO(test_data_obj['Body'].read()))
else:
    print(f"Query failed with status '{query_status_test}'")

Query failed with status 'FAILED'


In [10]:
test_location_id

'092390a2-b19f-4d52-9134-a07e30effb58'

In [12]:
s3_test_location = f"{s3_path}{test_location_id}.csv"
test_data_obj = s3.get_object(Bucket=bucket_name, Key=s3_test_location)
df_test = pd.read_csv(io.BytesIO(test_data_obj['Body'].read()))

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [4]:
# Access Queried Test Data

bucket_name = 'aai-540-final-data'
s3_test_query = 'athena/results/f4cf6706-1ba6-4538-ab86-13c8137facb5.csv'
test_data_obj = s3.get_object(Bucket=bucket_name, Key = s3_test_query)
df_test = pd.read_csv(io.BytesIO(test_data_obj['Body'].read()))

In [5]:
df_test_new = df_test[['text', 'emotions']]
del(df_test)
df_test_new.head()

,text,emotions
0,WE can't always have what we want. I see them ...,sadness
1,"I work 12 hour shifts, and agree. Tea is life-...",neutral
2,I don't even know who he is but I think he app...,neutral
3,COUNT YOUR CALORIES. EAT ABOVE YOUR TDEE EVERY...,neutral
4,Federal employee here. My amazing tenant offer...,affectionate


In [6]:
#Retrieve Model
role = sagemaker.get_execution_role()
s3 = boto3.client('s3')
model_dir = 'models/'
tar_file = 'base_model.tar.gz'


model_name = f"AIEmotion-base-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_data = f's3://{bucket_name}/{model_dir}{tar_file}'
tensorflow_version = '2.6.3'
transformers_version='4.17.0'
py_version = 'py38'
huggingface_model = HuggingFaceModel(model_data=model_data,
                                     role=role,
                                     transformers_version=transformers_version,
                                     tensorflow_version=tensorflow_version,
                                     py_version=py_version,
                                     entry_point='inference.py'
                                   )

In [7]:
endpoint_name = f"AIEmotion-base-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.c5.large',
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = AIEmotion-base-model-quality-monitor-2024-02-16-2146
-------!

In [8]:
# Predictions as baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket_name}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket_name}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data
Baseline results uri: s3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/baselining/results


In [9]:
# Grab a sample of the dataset
df_test_subset = df_test_new.sample(500)
df_test_subset = df_test_subset.reset_index(drop=True)

In [10]:
def calculate_predictions(results):
    df_results = pd.DataFrame(columns = ['pred_labels', 'probabilities'])
    for result in results:
        res = json.loads(result[0])
        pred_labels = list(res.keys())
        prob = list(res.values())
        row = {'pred_labels': pred_labels, 'probabilities': prob}
        df_results = df_results.append(row, ignore_index = True)
    return df_results

In [11]:
predictions = df_test_subset.apply(lambda x: predictor.predict({'text': x['text']}), axis = 1)
predictions

0      [{"neutral": 0.9746280312538147, "anger": 0.01...
1      [{"neutral": 0.671329915523529, "surprised": 0...
2      [{"happy": 0.9748302102088928, "affectionate":...
3      [{"affectionate": 0.45750537514686584, "happy"...
4      [{"neutral": 0.4965571463108063, "affectionate...
                             ...                        
495    [{"optimistic": 0.9062561988830566, "neutral":...
496    [{"happy": 0.5446568131446838, "affectionate":...
497    [{"happy": 0.8061304688453674, "neutral": 0.15...
498    [{"neutral": 0.41303831338882446, "happy": 0.3...
499    [{"surprised": 0.9695835709571838, "happy": 0....
Length: 500, dtype: object

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df_preds = calculate_predictions(predictions)
df_preds

,pred_labels,probabilities
0,"[neutral, anger, sad]","[0.9746280312538147, 0.0172532107681036, 0.002..."
1,"[neutral, surprised, anger]","[0.671329915523529, 0.17627429962158203, 0.055..."
2,"[happy, affectionate, sad]","[0.9748302102088928, 0.006392994895577431, 0.0..."
3,"[affectionate, happy, optimistic]","[0.45750537514686584, 0.4294200539588928, 0.07..."
4,"[neutral, affectionate, anger]","[0.4965571463108063, 0.39938440918922424, 0.02..."
...,...,...
495,"[optimistic, neutral, surprised]","[0.9062561988830566, 0.051451414823532104, 0.0..."
496,"[happy, affectionate, optimistic]","[0.5446568131446838, 0.4242367148399353, 0.014..."
497,"[happy, neutral, anger]","[0.8061304688453674, 0.15399974584579468, 0.02..."
498,"[neutral, happy, surprised]","[0.41303831338882446, 0.36112791299819946, 0.1..."


In [13]:
# Extract the first element of each tuple
df_preds['pred_labels'] = df_preds['pred_labels'].apply(lambda x: x[0])
df_preds['probabilities'] = df_preds['probabilities'].apply(lambda x: x[0])
df_preds

,pred_labels,probabilities
0,neutral,0.974628
1,neutral,0.671330
2,happy,0.974830
3,affectionate,0.457505
4,neutral,0.496557
...,...,...
495,optimistic,0.906256
496,happy,0.544657
497,happy,0.806130
498,neutral,0.413038


In [14]:
# Concat datasets
validate_dataset = pd.concat([df_test_subset, df_preds], ignore_index=True, axis = 1)
validate_dataset = validate_dataset.rename(columns = {0: 'text', 
                                                      1: 'true_label', 
                                                      2: 'pred_label', 
                                                      3: 'probability'})

validate_dataset['pred_label'] = validate_dataset['pred_label'].replace({'sad': 'sadness', 'surprised': 'surprise'})
validate_dataset

,text,true_label,pred_label,probability
0,[NAME] throws the puck to the slot for a cutti...,neutral,neutral,0.974628
1,Looks like the other driver to the right was p...,happy,neutral,0.671330
2,Thank you & my heartfelt condolences. Snap in ...,happy,happy,0.974830
3,Nice glad to hear man!,affectionate,affectionate,0.457505
4,What a unit,affectionate,neutral,0.496557
...,...,...,...,...
495,I hope google does this so that everyone would...,optimistic,optimistic,0.906256
496,I love how happy he looks while listening to t...,affectionate,happy,0.544657
497,Not even going to open the link. Already know ...,happy,happy,0.806130
498,Well...to be fair then. :),happy,neutral,0.413038


In [15]:
# Export to CSV
validate_dataset_name = 'validate_dataset.csv'
validate_dataset.to_csv(validate_dataset_name, index=False) 

In [16]:
# Upload baseline
baseline_dataset_uri = S3Uploader.upload(f"{validate_dataset_name}", baseline_data_uri)
baseline_dataset_uri

's3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

## Setup Baseline

In [20]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat



emotions_base_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

# Name of the model quality baseline job
baseline_job_name = f"AIEmotions-base-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"


# Execute the baseline suggestion job.
job = emotions_base_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="MulticlassClassification",
    inference_attribute="pred_label",
    probability_attribute="probability",
    ground_truth_attribute="true_label",
)
job.wait(logs=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name AIEmotions-base-model-baseline-job-2024-02-16-2156



Job Name:  AIEmotions-base-model-baseline-job-2024-02-16-2156
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://aai-540-final-data/sagemaker/AIEmotions-ModelQualityMonitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
............................................................................!

In [22]:
baseline_job = emotions_base_model_quality_monitor.latest_baselining_job

In [23]:
multiclass_metrics = baseline_job.baseline_statistics().body_dict["multiclass_classification_metrics"]
pd.json_normalize(multiclass_metrics).T

,0
confusion_matrix.disgust.disgust,6.000000
confusion_matrix.disgust.surprise,0.000000
"confusion_matrix.disgust._we'll_have_more_occasions_of_hearing_it_again_though_:)""",0.000000
confusion_matrix.disgust.happy,0.000000
confusion_matrix.disgust.optimistic,0.000000
...,...
weighted_f0_5_best_constant_classifier.standard_deviation,0.004077
weighted_f1_best_constant_classifier.value,0.117859
weighted_f1_best_constant_classifier.standard_deviation,0.005048
weighted_f2_best_constant_classifier.value,0.179094


In [24]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["multiclass_classification_constraints"]).T

,threshold,comparison_operator
accuracy,0.588,LessThanThreshold
weighted_recall,0.588,LessThanThreshold
weighted_precision,0.595712,LessThanThreshold
weighted_f0_5,0.589928,LessThanThreshold
weighted_f1,0.585759,LessThanThreshold
weighted_f2,0.585843,LessThanThreshold


## Setup continuous model monitoring

In [28]:
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)


def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "test-dataset-input-cols.csv")
        except session.sagemaker_runtime_client.exceptions.ValidationError:
            pass


thread = Thread(target=invoke_endpoint_forever)
thread.start()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/tmp/ipykernel_201/75971705.py", line 19, in invoke_endpoint_forever
  File "/tmp/ipykernel_201/75971705.py", line 2, in invoke_endpoint
  File "/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 282, in _modified_open
    return io_open(file, *args, **kwargs)
FileNotFoundError: [Errno 2] No such file or directory: 'test-dataset-input-cols.csv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_201/75971705.py", line 20, in invoke_endpoint_forever
AttributeError: module 'sagemaker.session' has no attribute 'sagemaker_runtime_client'


In [ ]:
print("Waiting for captures to show up", end="")
for _ in range(120):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

In [ ]:
print("\n".join(capture_file[-3:-1]))

In [ ]:
print(json.dumps(capture_record, indent=2))

In [29]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: AIEmotion-base-model-quality-monitor-2024-02-16-2146
INFO:sagemaker:Deleting endpoint with name: AIEmotion-base-model-quality-monitor-2024-02-16-2146


In [30]:
emotions_base_model_quality_monitor.delete_monitoring_schedule()
sleep(60)  # actually wait for the deletion


Deleting Monitoring Schedule with name: None


ParamValidationError: Parameter validation failed:
Invalid type for parameter MonitoringScheduleName, value: None, type: <class 'NoneType'>, valid types: <class 'str'>